<a href="https://colab.research.google.com/github/uteyechea/crime-prediction-using-artificial-intelligence/blob/master/temporal_correlation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import gc
import sys

import pandas as pd
from scipy import stats

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

path='/content/drive/My Drive/Colab Notebooks/crime_prediction'

sys.path.append(path+'/libs')
import temporal_correlation as tc


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
prediction_file_path=os.path.join(path,'data','prediction','series_sample.txt')
prediction_file=pd.read_csv(prediction_file_path,sep=',')
print(prediction_file.shape)

(2555, 1)


In [3]:
prediction_file.iloc[:,0] = pd.to_numeric(prediction_file.iloc[:,0], errors='coerce') #Remove non float values, substitute them with NAN

In [4]:
prediction_file=prediction_file.dropna()

In [5]:
prediction_file.shape

(2527, 1)

In [6]:
prediction_file.isnull().values.any()

False

In [7]:
training_data_path=os.path.join(path,'data','training','theft2.csv')
training_file=pd.read_csv(training_data_path,sep=',')

In [8]:
training_file.isnull().values.any()

False

In [9]:
type(prediction_file)

pandas.core.frame.DataFrame

In [10]:
type(training_file)

pandas.core.frame.DataFrame

In [11]:
prediction_file.shape

(2527, 1)

In [12]:
training_file.shape

(892, 1)

In [13]:
prediction_file

,-0.0.791945932798457
0,0.466208
1,-0.791946
2,0.969469
4,0.969469
5,-0.037054
...,...
2550,-0.540315
2551,0.717838
2552,0.466208
2553,0.717838


In [107]:
def series_window(dataframe,end,periods,column):
  if type(column)==str:
    column=dataframe.columns.get_loc(column)
  return dataframe.iloc[end-periods:end,column]

def series_correlation(series1,series2):
  #assert something?
  series1=series1.reset_index(drop=True) #Better find a way to simplify this procedure
  series2=series2.reset_index(drop=True) #Better find a way to simplify this procedure
  ro=series1.corr(series2) 
  return ro 

def correlated_series_timestamp(dataframe1,dataframe2,end,periods,column,min_correlation):
  timestamps=[]
  series1=series_window(dataframe1,end,periods,column)
  for epoch in reversed(range(2*periods,len(dataframe2))):
    series2=series_window(dataframe2,epoch-periods,periods,column)
    try:
      correlation=series_correlation(series1,series2)
      #print(correlation)
      if correlation >= min_correlation:
        #record datetime value. We will use this datetime value to generate all sequences that will go as input to the RNN
        #print(dataframe.index[epoch]) 
        timestamps.append(dataframe2.index[epoch])
    except:
      print('Can not compute correlation at index ',str(epoch))    
  return timestamps

def correlated_series_sequence(dataframe1,dataframe2,end,periods,column,min_correlation):
  timestamps=correlated_series_timestamp(dataframe1,dataframe2,end,periods,column,min_correlation)
  input={}
  output={}
  
  if type(column)==str:
    column=dataframe2.columns.get_loc(column)

  for index,timestamp in enumerate(timestamps):
    print(index)
    try:
      input[index]=dataframe2.iloc[timestamp-periods:timestamp,column]
      output[index]=dataframe2.iloc[timestamp:timestamp+periods,column]
    except:
      print('Missing index in DataFrame2')
  return input,output      

def get_IO_series(dataframe1,dataframe2,end,periods,column,min_correlation):
  timestamps=correlated_series_timestamp(dataframe1,dataframe2,end,periods,column,min_correlation)
  input,output=correlated_series_sequence(dataframe1,dataframe2,end,periods,column,min_correlation)
  #return input,output
  sequence={}
  for index,timestamp in enumerate(timestamps): #Change from input to timestamp sequence
    try:
      sequence[index]= input[index].append(output[index])
      sequence[index]= sequence[index].reset_index(drop=True)
    except:
      print('Error with key',str(timestamp))
  return sequence      

def save_to_file(sequence,file_path):
  with open(file_path,'a') as file:
    for timestamp in sequence:
      sequence[timestamp].to_csv(file,mode='a',header=False,index=False)
      file.write('\n')  


In [108]:
sequence=get_IO_series(training_file,prediction_file,end=10,periods=10,column=0,min_correlation=0.85)

0
1
2


In [109]:
type(sequence)

dict

In [110]:
seq=pd.DataFrame(sequence)

In [111]:
seq

,0,1,2
0,2.476231,0.214577,-0.791946
1,1.472730,1.221100,-0.540315
2,0.717838,1.472730,0.969469
3,0.965469,1.724361,-0.288685
4,0.969469,0.214577,0.466208
5,0.214577,0.214577,-0.791946
6,1.221093,1.724361,-0.791946
7,0.466208,0.969469,-1.043577
8,0.466208,0.717838,-0.288685
9,0.969469,-0.037054,-0.288685


In [112]:
import plotly.express as px

In [150]:
px.line(seq,x=seq.index,y=0)

In [151]:
px.line(seq,x=seq.index,y=1)

In [152]:
px.line(seq,x=seq.index,y=2)

In [159]:
sequence=get_IO_series(training_file,training_file,end=10,periods=10,column=0,min_correlation=0.95)

0


In [160]:
seq=pd.DataFrame(sequence)

In [161]:
px.line(seq,x=seq.index,y=0)

In [158]:
px.line(seq,x=seq.index,y=1)